In [43]:
import numpy as np
import math
import random

import gym
import gym_maze

In [44]:
#!pip install pygame

In [46]:
env = gym.make("maze-random-10x10-v0")
env.reset()
env.render()

array([[[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       [[  0,   0,   0],
        [ 20,  20, 159],
        [ 20,  20, 159],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [  0,   0,   0]],

       [[  0,   0,   0],
        [ 20,  20, 159],
        [ 20,  20, 159],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [  0,   0,   0]],

       ...,

       [[  0,   0,   0],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [159,  20,  20],
        [159,  20,  20],
        [  0,   0,   0]],

       [[  0,   0,   0],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [159,  20,  20],
        [159,  20,  20],
        [  0,   0,   0]],

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]]

In [47]:
import pandas as pd
import numpy as np
import time

In [22]:
# env.reset()

# for _ in range(1000):
#     env.render()

# env.close()

In [48]:
def init_env():
    start = (0,0)
    terminal = (3,2)
    hole = (2,1)
    env = np.array([["_ "]*10]*10)
    env[terminal] = "$ "
    env[hole] = "# "
    env[start] = "L "
    interaction = " "

    for i in env:
        interaction += "".join(i) + "\n"
    print(interaction)

init_env()

 L _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ 
_ # _ _ _ _ _ _ _ _ 
_ _ $ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ 



In [49]:
def init_q_table():
    #Q-Table
    actions = np.array(['up', 'down', 'left', 'right'])
    q_table = pd.DataFrame(
        np.zeros((100, len(actions))), columns = actions
    )
    return q_table

In [50]:
def act_choose(state, q_table, epsilon):
    '''
    state
    q_table
    epsilon

    action
    '''

    state_act = q_table.iloc[state, :]
    actions = np.array(['up', 'down', 'left', 'right'])

    if np.random.uniform() > epsilon or state_act.all() == 0:
        action = np.random.choice(actions)
    else:
        action = state_act.idxmax()
    return action 


In [51]:
def env_feedback(state, action, hole, terminal):
    reward = 0.0
    end = 0 

    a,b = state
    
    if action == "up":
        a -= 1
        if a < 0:
            a = 0
        next_state = (a, b)
    elif action == "down":
        a += 1
        if a >= 4:
            a = 3
        next_state = (a, b)
    elif action == "left":
        b -=1
        if b < 0:
            b = 0
        next_state = (a, b)
    elif action == "right":
        b += 1
        if b >= 4:
            b = 3
        next_state = (a,b)

    if next_state == terminal:
        reward = 10.0
        end = 2
    elif next_state == hole:
        reward = -10.0
        end = 1
    else:
        reward = -1.0
    return next_state, reward, end

In [52]:
def update_q_table(q_table, state, action, next_state, terminal, gamma, alpha, reward):
    x, y = state 
    next_x, next_y = next_state 
    q_original = q_table.loc[x * 4 + y, action] 
    if next_state != terminal: 
        q_predict = reward + gamma * q_table.iloc[next_x * 4 + next_y].max()
    else:
        q_predict = reward 
        
    q_table.loc[x * 4 + y, action] = (1 - alpha) * q_original + alpha * q_predict
    
    return q_table 

In [53]:
def show_state(end, state, episodes, step, q_table):
    # Your implementation to visualize or print the state
    print(f"End: {end}, State: {state}, Episodes: {episodes}, Steps: {step}, Q-table: {q_table}")

In [54]:
def q_learning(max_episodes, alpha, gamma, epsilon): 
    q_table = init_q_table() 
    terminal = (3, 2) 
    hole = (2, 1) 
    episodes = 0 
    step = 0
    
    while episodes <= max_episodes: 
        slep = 0 
        state = (0, 0) 
        end = 0 
        show_state(end, state, episodes, step, q_table) 

        while end == 0: 
            x, y = state 
            act = act_choose(x * 4 + y, q_table, epsilon) 
            next_state, reward, end = env_feedback(state, act, hole, terminal) 
            q_table = update_q_table( 
                q_table, state, act, next_state, terminal, gamma, alpha, reward 
                ) #q-table 
            state = next_state 
            step += 1 
            show_state(end, state, episodes, step, q_table) 
        if end == 2: 
            episodes += 1 